In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy import asarray

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

print(tf.__version__)

2.0.0


In [2]:
# check if GPU is used correctly
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [ ]:
# set directories
test_dir = os.path.join("D:/", "# Studie/# MSc Data Science/Applied Machine Learning/Kaggle_Project", "test_set")
train_dir = os.path.join("D:/", "# Studie/# MSc Data Science/Applied Machine Learning/Kaggle_Project", "train_set")

In [ ]:
'''
1. This part shows the code of our first approach, our self-build CNN.
'''

In [ ]:
# parameters of the model
batch_size = 32
epochs = 20
IMG_HEIGHT = 200
IMG_WIDTH = 200

In [ ]:
# train image loader
train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           zoom_range=0.1,
                                           rotation_range=30,
                                           width_shift_range=0.2,
                                           height_shift_range=0.2,
                                           horizontal_flip=True,
                                           shear_range=0.1,
                                           fill_mode='nearest',
                                           validation_split=0.2) # Generator for our training data

In [ ]:
# create train and validation datasets
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='training')

val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical',
                                                           subset='validation')

In [ ]:
sample_training_images, labels = next(train_data_gen)

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# plot 5 training images
plotImages(sample_training_images[:5])

In [ ]:
# create a dictionary with the labels versus the indices
class_label_dict = train_data_gen.class_indices

# print three sample images to check if the indices are correctly tied to the labels
for i in range(0,3):
    image = sample_training_images[i]
    label = labels[i]
    plt.imshow(image)
    data = asarray(image)
    print(data.shape)
    prediction = np.argmax(label)
    print("Given prediction", prediction)
    for key, value in class_label_dict.items():
        if value == prediction:
            print("Actual key", key)
    plt.show()

In [ ]:
# model architecture
model = Sequential([
    Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),

    Flatten(),
    
    Dense(512, activation='relu'),
        
    Dense(80, activation='softmax')
])

In [ ]:
# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# this trains the model, we added learning rate reducer when the model hits a plateau
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch= train_data_gen.samples // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps= val_data_gen.samples // batch_size,
    callbacks=[ReduceLROnPlateau(factor=.5, patience=4, verbose=1)]
)

In [ ]:
# this function plots the train accuracy and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(sample_training_images, labels, verbose=2)

In [ ]:
# import test dataset
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=test_dir,
                                                           shuffle=False,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode="categorical")

In [ ]:
# get predictions for the test set
test_generator.reset()
pred=model.predict_generator(test_generator)
predictions= np.argmax(pred, axis=1)
filenames = test_generator.filenames
filenames1 = [i[9:] for i in filenames]

In [ ]:
# initalize the submission dataframe
submission_df = pd.DataFrame( {'img_name': filenames1,'prediction': predictions, 'label': " "})

# get labels from the predictions
for index, row in submission_df.iterrows():
    prediction = row['prediction']
    for key, value in class_label_dict.items():
        if prediction == value:
            submission_df.at[index, 'label'] = key

# export the submission_df to upload to kaggle
submission_df = submission_df.drop(columns="prediction")
submission_df.to_csv('submission.csv', index=False)

In [ ]:
'''
2. This part of the code will be our Ensemble model

'''

In [ ]:
# parameters of the model
batch_size = 32
epochs = 20
IMG_HEIGHT = 200
IMG_WIDTH = 200
n_members = 20 #number of ensemble models to make

In [ ]:
# makes n_member number of CNNs with new data and saves every model to file

class_label_dict = []   # global variable for class label indices dictionary

for _ in range(n_members):
    
    # every iteration resample images from the train and validation set
    val_data_gen.reset()
    train_data_gen.reset()
    
    # save class indices for prediction
    class_label_dict = train_data_gen.class_indices
    
    # define a model
    model = Sequential([
        Conv2D(32, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Dropout(0.2),

        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),

        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPooling2D(),

        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Dropout(0.2),

        Flatten(),

        Dense(512, activation='relu'),

        Dense(80, activation='softmax')
    ])
    
    # compile the model
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    # train the model
    history = model.fit_generator(
        train_data_gen,
        steps_per_epoch= train_data_gen.samples // batch_size,
        epochs=epochs,
        validation_data=val_data_gen,
        validation_steps= val_data_gen.samples // batch_size,
        callbacks=[ReduceLROnPlateau(factor=.5, patience=4, verbose=1)]
    )
    # save the model to file for later
    model.save('good_model'+ str(_)+ '.h5')

In [ ]:
# craete dataframe for model predictions
submission_df = pd.DataFrame( {'img_name': [i[9:] for i in test_generator.filenames]})

In [ ]:
# function that takes the predictions of a model, and transforms it into the correct label
def get_label_value(prediction):
    pred = np.argmax(prediction, axis=1)
    labels = []
    for i in pred:
        for key, value in class_label_dict.items():
            if i == value:
                labels.append(key)
    return labels

In [ ]:
# for every model, predict the labels for all test images and append to the submission_df
for i in range(n_members):
    
    test_generator.reset()
    
    filename = 'model_' + str(i) + '.h5'
    current_model = tf.keras.models.load_model(filename)
    
    pred=current_model.predict_generator(test_generator)
    print('model_'+str(i))
    submission_df[filename] = get_label_value(pred)

In [ ]:
# get the mode of all predictions
submission_df['label'] = submission_df.filter(like='model').mode(axis=1).iloc[:, 0]

# write file with submission
submission_df[['img_name', 'label']].to_csv('submission.csv', index=False)

In [ ]:
'''
3. This part of code will show how we used ResNet for our transfer learning model
'''

In [ ]:
# parameters of the model
batch_size = 16 # larger batch size did not fit the GPU unfortunately
epochs = 20
IMG_HEIGHT = 224 
IMG_WIDTH = 224

In [ ]:
# define how the images have to be pre-processed
train_image_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
                                           validation_split=0.2) # Generator for our training data

# generator for our training data
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                       directory=train_dir,
                                                       shuffle=True,
                                                       target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                       class_mode='categorical',
                                                       subset='training')

# generator for our validation data
val_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                       directory=train_dir,
                                                       shuffle=True,
                                                       target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                       class_mode='categorical',
                                                       subset='validation')

# save class indices for prediction
class_label_dict = train_data_gen.class_indices

In [ ]:
# our base model is the ResNet50 model
base_model = tf.keras.applications.ResNet50(input_shape=(IMG_HEIGHT, IMG_WIDTH ,3),
                                               include_top=False,
                                               weights='imagenet')

# global average pooling layer between the pre-trained network and the last dense layer
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

# dense layer that puts out the 80 classes
prediction_layer = tf.keras.layers.Dense(80, activation='softmax')

In [ ]:
# define the model
model = Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])
# unfreezing the last 50 layers (we have tried with and without unfreezing)
base_model.trainable=False
for layer in base_model.layers[150:]:
    layer.trainable = True

In [ ]:
# compiling the model
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

# train the model
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch= train_data_gen.samples // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps= val_data_gen.samples // batch_size
)

In [ ]:
# load test images
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
test_generator = test_datagen.flow_from_directory(batch_size=batch_size,
                                                           directory=test_dir,
                                                           shuffle=False,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode="categorical")


# create dataframe for model predictions
submission_df = pd.DataFrame( {'img_name': [i[9:] for i in test_generator.filenames]})

In [ ]:
# get predictions and use get_label_value function to get proper labels
pred=model.predict_generator(test_generator)
submission_df['label'] = get_label_value(pred)

In [ ]:
# write file with submission
submission_df[['img_name', 'label']].to_csv('submission.csv', index=False)